# Tarea Apache Spark
>Importa las librerías necesarias dónde sea necesario


In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=1b9bdfb2d209dd709d9454d47dd082fc3f019f044c32d395eab6554457f4ce09
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
import pandas as pd
from pyspark.sql import SparkSession

### SparkSession
>Crea un SparkSession para comenzar la tarea

In [4]:
# Configurar la aplicación Spark
app_name = "tarea"

# Crear una instancia de SparkSession
spark = SparkSession.builder.appName(app_name).getOrCreate()


### Crear un DataFrame
>Lee el csv datosTarea.csv, mételo a un DF y muéstralo.

In [6]:
df = spark.read.csv("/datosTarea.csv", header=True, inferSchema=True)
df.show()

+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------+-----------+------+
|Index|Organization Id|                Name|             Website|             Country|         Description|Founded|            Industry|Number of employees|Networth|stock_price|profit|
+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------+-----------+------+
|    1|FAB0d41d5b5d22c|         Ferrell LLC|  https://price.net/|    Papua New Guinea|Horizontal empowe...|   1990|            Plastics|               3498|  402269|         33| 12125|
|    2|6A7EdDEA9FaDC52|Mckinney, Riley a...|http://www.hall-b...|             Finland|User-centric syst...|   2015|Glass / Ceramics ...|               4952|  569480|         49| 12001|
|    3|0bFED1ADAE4bcC1|          Hester Ltd|http://sullivan-r...|          

### Filtro de datos
>Consigue todas las empresas que empiecen con 'M' y tengan entre 4000 y 7000 empleados. Sólo muestra los nombres y el número de empleados.

In [9]:
resp = df.filter((df['Name'].startswith('M')) & (df["Number of Employees"].between(4000, 7000))).select("Name", "Number of Employees")

resp.show()

+--------------------+-------------------+
|                Name|Number of Employees|
+--------------------+-------------------+
|Mckinney, Riley a...|               4952|
|       Mcintosh-Mora|               4389|
|     Mckenzie-Melton|               4589|
|          Massey LLC|               5004|
|        Mays-Preston|               5786|
+--------------------+-------------------+



>Consigue todos los países que no inicien con las letras 'b', 's' y 'm', pero que tampoco tengan un netword mayor a 500000. Muestra el nombre de la compañía, el país y el networth.

In [20]:
# Filtrar los países que no comienzan con 'b', 's' o 'm' y tienen un networth menor o igual a 500000
result_df = df.filter(
    (~col("Country").startswith("b")) &
    (~col("Country").startswith("s")) &
    (~col("Country").startswith("m")) &
    (col("Networth") <= 500000)
)

# Mostrar el resultado
result_df.select("Name", "Country", "Networth").show()

+--------------------+--------------------+--------+
|                Name|             Country|Networth|
+--------------------+--------------------+--------+
|         Ferrell LLC|    Papua New Guinea|  402269|
|      Holder-Sellers|        Turkmenistan|  105914|
|Keller, Campos an...|             Liberia|  329130|
|      Pacheco-Spears|              Sweden|   88435|
|         Harrell LLC|          Guadeloupe|  251274|
|         Jenkins Inc|        South Africa|  139725|
|Dickson, Richmond...|      Czech Republic|  359030|
|        Prince-David|    Christmas Island|  120289|
|         Rivas Group|           Australia|  477824|
|Sloan, Mays and W...|                Chad|   41975|
|Glass, Barrera an...|     Kyrgyz Republic|  300150|
|          Pineda-Cox|             Bolivia|  150880|
|Baker, Mccann and...|               Kenya|  188370|
|            Hahn PLC|             Belarus|  427224|
|Valentine, Fergus...|              Jersey|  412274|
|           Walls LLC|          Cape Verde|  1

### Funciones
Crea una función con @pandas_udf que que le reste a los profits la media en cada renglón. Crea una nueva columna que muestre los resultados.

In [37]:
def subtract_mean(pdf):
    return pdf.assign(profit_minus_mean=pdf['profit'] - pdf['profit'].mean())

### Grouping data
>Agrupa por industry y muestra cuáles son las empresas con el profit más alto. Muestra los primeros tres.

In [22]:
from pyspark.sql.functions import col, rank
from pyspark.sql.window import Window
from pyspark.sql.functions import desc

In [23]:
# Agrupar por industry y encontrar las empresas con el profit más alto
result_df = df.groupBy("Industry").agg({"Profit": "max"}).withColumnRenamed("max(Profit)", "max_profit")

# Ordenar por profit de manera descendente y mostrar los tres primeros
result_df = result_df.orderBy(desc("max_profit")).limit(3)

# Mostrar el resultado
result_df.show()

+--------------------+----------+
|            Industry|max_profit|
+--------------------+----------+
|  Legislative Office|     19363|
|Museums / Institu...|     19079|
|             Farming|     18850|
+--------------------+----------+



>Agrupa por industry y calcula el promedio de empleados que tienen

In [25]:
from pyspark.sql.functions import avg

In [26]:
# Agrupar por industry y calcular el promedio de empleados
result_df = df.groupBy("Industry").agg(avg("Number of Employees"))

# Mostrar el resultado
result_df.show()

+--------------------+------------------------+
|            Industry|avg(Number of Employees)|
+--------------------+------------------------+
|Primary / Seconda...|       6457.666666666667|
|     Broadcast Media|                  2589.0|
|           Wholesale|                  5010.0|
|Investment Manage...|                  3133.5|
|    Food / Beverages|                  9011.0|
|  Gambling / Casinos|                  4873.0|
|Logistics / Procu...|                  4155.0|
|            Maritime|                   769.0|
|            Wireless|                  6146.0|
|Education Management|                   339.0|
|       Arts / Crafts|                  2800.0|
|           Insurance|                  1215.0|
|  Financial Services|                  5157.0|
|Business Supplies...|                  9097.0|
|Consumer Electronics|                  5022.0|
|       Public Safety|                  5287.0|
|Information Techn...|                  3934.0|
|Civic / Social Or...|                  

### SQL
>Usando Spark SQL, obtén cuántas empresas se fundaron despúes del 2000.

In [10]:
df.createOrReplaceTempView("empresa_table")
query = """
    SELECT COUNT(*) as count_empresas
    FROM empresa_table
    WHERE Founded > 2000
"""

result = spark.sql(query)

result.show()

+--------------+
|count_empresas|
+--------------+
|            38|
+--------------+



### ML Regresión Lineal
>Con número de empleados, networth y stock price, obtén una predicción del profit a través de una regresión lineal.

In [28]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [35]:
# Ensamblar las características en un vector
assembler = VectorAssembler(
    inputCols=["Number of employees", "Networth", "stock_price"],
    outputCol="features"
)
df_assembled = assembler.transform(df)
# Seleccionar las columnas relevantes
data = df_assembled.select("features", "Profit")

# Dividir el conjunto de datos en entrenamiento y prueba
train_data, test_data = data.randomSplit([0.8, 0.2], seed=123)

# Configurar el modelo de regresión lineal
lr = LinearRegression(featuresCol="features", labelCol="Profit")

# Entrenar el modelo
lr_model = lr.fit(train_data)

# Realizar predicciones en el conjunto de prueba
predictions = lr_model.transform(test_data)

>Una vez que obtengas los resultados, a través del api de pandas, conviértelo en un pandas on spark DataFrame y pásalo a csv.

In [36]:
# Convertir el DataFrame de PySpark a un DataFrame de pandas
pandas_df = predictions.select("features", "profit", "prediction").toPandas()

# Guardar el DataFrame de pandas como un archivo CSV
pandas_df.to_csv("resultados.csv", index=False)